In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from skimage.color import rgb2hed
from sklearn.mixture import GaussianMixture
import skfuzzy as fuzz
from glob import glob
from tqdm import tqdm
from scipy.stats import entropy as calc_entropy
from skimage.segmentation import find_boundaries
from scipy.spatial.distance import directed_hausdorff

In [2]:
input_path = 'output_normalized/'
output_path = 'outputs/stain_segmentation/'
os.makedirs(output_path, exist_ok=True)

image_paths = sorted(glob(os.path.join(input_path, '*.tif')))


In [3]:
# Visualization: show original + all clustering masks side-by-side
def display_all_results(image, masks, titles):
    plt.figure(figsize=(20, 5))
    plt.subplot(1, len(masks) + 1, 1)
    plt.imshow(image)
    plt.title("Normalized Image")
    plt.axis('off')
    
    for i, (mask, title) in enumerate(zip(masks, titles)):
        plt.subplot(1, len(masks) + 1, i + 2)
        plt.imshow(mask, cmap='jet')
        plt.title(title)
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
# Clustering method
def stain_clustering(img_rgb, method='kmeans', n_clusters=2):
    hed = rgb2hed(img_rgb)
    eosin = hed[:, :, 1]
    nuclei = hed[:, :, 0]
    features = np.stack([nuclei.flatten(), eosin.flatten()], axis=1)

    if method == 'kmeans':
        model = KMeans(n_clusters=n_clusters, random_state=42).fit(features)
        labels = model.labels_
    elif method == 'fcm':
        cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(features.T, c=n_clusters, m=2, error=0.005, maxiter=1000)
        labels = np.argmax(u, axis=0)
    elif method == 'gmm':
        model = GaussianMixture(n_components=n_clusters, covariance_type='tied').fit(features)
        labels = model.predict(features)
    else:
        raise ValueError("Invalid clustering method")

    mask = labels.reshape(img_rgb.shape[:2])
    return mask


In [ ]:
# Entropy calculation
def compute_entropy(mask):
    hist, _ = np.histogram(mask, bins=np.arange(3), density=True)
    return calc_entropy(hist)

# Boundary Displacement Index calculation
def compute_bdi(mask1, mask2):
    b1 = find_boundaries(mask1, mode='thick')
    b2 = find_boundaries(mask2, mode='thick')

    y1, x1 = np.argwhere(b1).T
    y2, x2 = np.argwhere(b2).T

    if len(x1) == 0 or len(x2) == 0:
        return np.inf  # no boundaries

    d1 = directed_hausdorff(np.column_stack((x1, y1)), np.column_stack((x2, y2)))[0]
    d2 = directed_hausdorff(np.column_stack((x2, y2)), np.column_stack((x1, y1)))[0]

    return (d1 + d2) / 2  # symmetric BDI

In [ ]:
algorithms = ['kmeans', 'fcm', 'gmm']

for path in tqdm(image_paths):
    fname = os.path.basename(path).replace('.tif', '')
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    masks = {}
    metrics = {}

    # Run clustering and save masks + overlays
    for algo in algorithms:
        mask = stain_clustering(img, method=algo, n_clusters=2)
        masks[algo] = mask

        out_dir = os.path.join(output_path, algo)
        os.makedirs(out_dir, exist_ok=True)

        np.save(os.path.join(out_dir, f"{fname}_mask.npy"), mask)
        overlay = cv2.addWeighted(img, 0.6, cv2.applyColorMap((mask * 127).astype(np.uint8), cv2.COLORMAP_JET), 0.4, 0)
        cv2.imwrite(os.path.join(out_dir, f"{fname}_overlay.png"), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    # Compute metrics
    for algo, mask in masks.items():
        ent = compute_entropy(mask)
        # Use kmeans mask as reference for BDI except for kmeans itself
        bdi = compute_bdi(mask, masks['kmeans']) if algo != 'kmeans' else 0
        metrics[algo] = {'entropy': ent, 'bdi': bdi}

    # Ranking by entropy then BDI (lower better)
    ranking = sorted(metrics.items(), key=lambda x: (x[1]['entropy'], x[1]['bdi']))

    # Print results
    print(f"\nImage: {fname}")
    for algo, vals in metrics.items():
        print(f"  {algo.upper():6s} - Entropy: {vals['entropy']:.4f}, BDI: {vals['bdi']:.4f}")
    print("Ranking:")
    for i, (algo, _) in enumerate(ranking, 1):
        print(f"  {i}. {algo.upper()}")

    # Visualization for images containing 'TCGA-AC' in filename (optional)
    if 'TCGA-AC' in fname:
        display_all_results(img, [masks[a] for a in algorithms], [a.upper() for a in algorithms])

print("\nStain component clustering, evaluation, and ranking completed.")